In this project, I will work with a table extracted from the World Factbook database.

The World Factbook, also known as the CIA World Factbook, is a reference resource produced by the Central Intelligence Agency (CIA) with almanac-style information about the countries of the world.

Data Source: https://www.cia.gov/library/publications/download/download-2015/index.html

In [3]:
!conda install -yc conda-forge ipython-sql\

Solving environment: done

# All requested packages already installed.



In [4]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

First of all, I am going to display the first 5 rows of the table in order to get myself familiar with the data.

In [5]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


As we can see, there are 11 columns in the table - "Facts":

1. id
   * This could be a key to join with other tables in the database.
2. code - ISO Alpha-2 country codes
3. name - The name of the country.
4. area- The country's total area (both land and water).
5. area_land - The country's land area in square kilometers.
6. area_water - The country's waterarea in square kilometers.
7. population - The country's population.
8. population_growth- The country's population growth as a percentage.
9. birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
10. death_rate - The country's death rate, or the number of death a year per 1,000 people.
11. migration_rate - The country's migration rate, or the difference between the number of immigrants and the number of emigrants per year.


I would grouped into two categories that describe a country's demographics:

1. People/Population(Population, Growth, Birth & Death Rate, Migration Rate)
2. Area(Land, Water, Total)


Now I am going to ask some questions and compute summary statistics:

1. Population

1-1. Which country has the largest population?
1-2. Is the country with the largest population also the most densely populated?
1-3. Does this country has the highest birth rate?
1-4. What is its ratio of birth/rate rate?

In [6]:
%%sql
SELECT MAX(population) AS Max_Pop, 
       MIN(population) AS Min_Pop, 
       MAX(population_growth) AS Max_PopGrow, 
       MIN(population_growth) AS Min_PopGrow
  FROM facts;

 * sqlite:///factbook.db
Done.


Max_Pop,Min_Pop,Max_PopGrow,Min_PopGrow
7256490011,0,4.02,0.0


It is interesting that: 

1. The maximum population shown here is over 7 million, which is unlikely for a country because earth's population is about 7.5 million.
2. The minimum population is 0. 

So let's look into this:

In [7]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


In [8]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population)
                        FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


It turns out that: 

1. The table contains a row for the whole world, which explains why the largest population is over 7 million
2. The row where population is 0 is for Antartica(https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html), Earth's southernmost continent where there is no inhabitant. 

Moving forward, I am going to exclude the row for the "world".

In [9]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                       WHERE name <> 'World'); 

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


The result tells us that China(https://www.cia.gov/library/publications/the-world-factbook/geos/ch.html) has the largest population.

In [10]:
%%sql
SELECT code, name, population, area_land, population/area_land AS pop_density
  FROM facts
 WHERE pop_density == (SELECT MAX(population/area_land)
                         FROM facts);

 * sqlite:///factbook.db
Done.


code,name,population,area_land,pop_density
mc,Macau,592731,28,21168


Macau, also spelled Macao, and officially the Macao Special Administrative Region of the People's Republic of China, is a city in the western Pearl River Delta by the South China Sea.(https://www.cia.gov/library/publications/the-world-factbook/geos/mc.html)

Although Macau which is part of China has the highest population density, it does not mean that China is the most densely populated country.

Now I want to see the top 10 most densely populated countries/territories/regions to understand better.

In [12]:
%%sql
SELECT name, population, area_land, population/area_land AS pop_density
  FROM facts
 WHERE pop_density
 ORDER BY pop_density DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


name,population,area_land,pop_density
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,687,8259
Hong Kong,7141106,1073,6655
Gaza Strip,1869055,360,5191
Gibraltar,29258,6,4876
Bahrain,1346613,760,1771
Maldives,393253,298,1319
Malta,413965,316,1310
Bermuda,70196,54,1299


It look like that Monaco(https://www.cia.gov/library/publications/the-world-factbook/geos/mn.html) is the most densely populated country with a population density of 211,68 people per square kilometer.

I also noticed that Hong Kong(https://www.cia.gov/library/publications/the-world-factbook/geos/print_hk.html) is the 4th most densely populated territory that is also part of People's Republic of China. 

In [95]:
%%sql
SELECT *
  FROM facts
 WHERE birth_rate = (SELECT MAX(birth_rate)
                       FROM facts); 

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56


As the most populated country, China does not have the highest birth rate. 

According to Statista.com, the fertility rate in Niger(https://www.cia.gov/library/publications/the-world-factbook/geos/ng.html) was estimated to be 6.49 children per woman, in 2017.(https://www.statista.com/statistics/262884/countries-with-the-highest-fertility-rates/)

In [13]:
%%sql
SELECT population_growth, birth_rate/death_rate
  FROM facts
 WHERE name = 'China'; 

 * sqlite:///factbook.db
Done.


population_growth,birth_rate/death_rate
0.45,1.658698539176627


This tells us: there were about 1.7 births for every death in China as of 2015.

In [17]:
%%sql
SELECT AVG(population_growth), (SELECT population_growth
                                  FROM facts
                                 WHERE name = 'China') AS China_population_growth
  FROM facts
 WHERE name <> 'World'; 

 * sqlite:///factbook.db
Done.


AVG(population_growth),China_population_growth
1.2014893617021278,0.45


China has a population growth rate that is 62.5% below the average population growth rate of the world. 

Its slow population growth and low birth rate could be explained by the one-child policy.


2. Area

2-1. Which country is the largest country by area?
2-2. Does this country hold a large population?
2-3. What is its ratio of land to water?

In [122]:
%%sql
SELECT *
  FROM facts
 WHERE name <> 'world'
 ORDER BY area DESC
 LIMIT 20;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69
32,ca,Canada,9984670,9093507,891163,35099836,0.75,10.28,8.42,5.66
186,us,United States,9826675,9161966,664709,321368864,0.78,12.49,8.15,3.86
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44
24,br,Brazil,8515770,8358140,157630,204259812,0.77,14.46,6.58,0.14
9,as,Australia,7741220,7682300,58920,22751014,1.07,12.15,7.14,5.65
197,ee,European Union,4324782,None,None,513949445,0.25,10.2,10.2,2.5
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04
7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.0
87,kz,Kazakhstan,2724900,2699700,25200,18157122,1.14,19.15,8.21,0.41


Russia(https://www.cia.gov/library/publications/the-world-factbook/geos/rs.html) is the largest country in the world by area.

In [40]:
%%sql
SELECT AVG(population)/(SELECT population
                          FROM facts
                         WHERE name = 'Russia')
       AS Russia_to_Avg_Pop
  FROM facts
 WHERE name <> 'world';

 * sqlite:///factbook.db
Done.


Russia_to_Avg_Pop
0.4359871039388491


Despite its large area, Russia has a relatively small total population. 
However, its population is still rather large in numbers in comparison to those of other countries.

In [41]:
%%sql
SELECT name, area, area_land, area_water,
       Round(area_land/ Cast(area AS FLOAT), 2) AS Russian_land_pct,
       Round(area_water/ Cast(area AS FLOAT), 2) AS Russian_water_pct,
       area_land/area_water AS Russia_land_to_water
  FROM facts
 WHERE name = 'Russia';

 * sqlite:///factbook.db
Done.


name,area,area_land,area_water,Russian_land_pct,Russian_water_pct,Russia_land_to_water
Russia,17098242,16377742,720500,0.96,0.04,22


About 96% of Russia is covered by land, and about 4% is covered by water.

Now I am curious about which country/territory/region has the highest percentage of land and of water.

In [35]:
%%sql
SELECT name, area, area_land, area_water,
       MAX(Round(area_land/ Cast(area AS FLOAT), 2)) AS Max_land_pct
  FROM facts
 WHERE name <> 'world';

 * sqlite:///factbook.db
Done.


name,area,area_land,area_water,Max_land_pct
Afghanistan,652230,652230,0,1.0


Afghanistan(https://www.cia.gov/library/publications/the-world-factbook/geos/af.html) is a landlocked mountainous country located within Central and South Asia.

In [33]:
%%sql
SELECT name, area, area_land, area_water,
       MIN(Round(area_land/ Cast(area AS FLOAT), 4)) AS Min_land_pct
  FROM facts
 WHERE name <> 'world';

 * sqlite:///factbook.db
Done.


name,area,area_land,area_water,Min_land_pct
British Indian Ocean Territory,54400,60,54340,0.0011


The land area of British Indian Ocean Territory(https://www.cia.gov/library/publications/the-world-factbook/geos/print_io.html) is about one-third the size of Washington, DC.

Some facts learned from above:

1. Population

1-1. China is the largest country by population.
1-2. Monaco is the most densely poppulated country.
1-3. China doesn't have the highest birth rate. Niger has the highest birth rate.
1-4. China has a slow population growth rate.


2. Area

2-1. Russia is the largest country by area.
2-2. Russia holds a relatively small population. It is not densely populated.
2-3. Russia's ratio of land to water is 22:1. 
2-4. Afghanistan has 0% of water by area, because it is a landlocked country.
2-5. British Indian Ocean Territory is mostly covered by water(99%).